### Import libraries

In [12]:
import cv2
import numpy as np

In [21]:
#Drwa the lines on image;lines has start and end point of lines
def draw_the_lines(image,lines):
    #Create distinct image for lines
    lines_image=np.zeros((image.shape[0],image.shape[1],3),dtype=np.uint8)
    #x and y coordinates for start and end points;blue line
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(lines_image,(x1,y1),(x2,y2),(255,0,0),thickness=3)
    
    #Merge image with lines;0.8 transparancy for image and 1 transparancy for lines image
    image_with_lines=cv2.addWeighted(image,0.8,lines_image,1,0.0)
    return image_with_lines
        

In [22]:
#Return the region of interest alone from image by creating mask and performing AND operator on image
def region_of_interest(image,region_points):
    #Replace the pixels of uninterested regions in image with 0
    mask=np.zeros_like(image)
    #Region of interested will be replaced by 255 in mask
    cv2.fillPoly(mask,region_points,255)
    #Perform AND operation b/w image and mask
    masked_image=cv2.bitwise_and(image,mask)
    return masked_image   

In [26]:
#Return the video with lanes detected
def get_detected_lanes(image):
    (height,width)=(image.shape[0],image.shape[1])
    #Convert image into gray scale
    gray_image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    #Edge detection kernel(Canny's algorithm);100 & 120 thresholds
    canny_image=cv2.Canny(gray_image,100,120)
    
    #Get the region of interest ie lower region of image as a triangle enclosing the lanes
    roi_vertices=[
        (0,height),#bottom left corner
        (width/2,height*0.65),#point below middle of image
        (width,height)#bottom right corner
    ]
    
    #We can get rid of unrelevant part of image and keep the lower triangle region
    cropped_image=region_of_interest(canny_image,np.array([roi_vertices],np.int32))
    
    #Use Hough algorithm to generate lines;angle is in radians instead of degrees ie 1degree=pi/180
    lines=cv2.HoughLinesP(cropped_image,rho=2,theta=np.pi/180,threshold=50,lines=np.array([]),minLineLength=40,maxLineGap=150)
    
    #Draw the lines on the image
    image_with_lines=draw_the_lines(image,lines)
    
    return image_with_lines

### Read and play the video

In [27]:
#Video is composed of multiple frames/images shown one after the other
video=cv2.VideoCapture('video/lane_detection_video.mp4')

In [28]:
while video.isOpened():
    is_grabbed,frame=video.read()
    #Check if end of video
    if not is_grabbed:
        break
    
    frame=get_detected_lanes(frame)
    cv2.imshow('Lane Detection Video',frame)
    #To introduce a wait time between frames for viewing video
    cv2.waitKey(20)

video.release()
cv2.destroyAllWindows()
        